<a href="https://colab.research.google.com/github/Honsl/Llama2-Prompt-Engineering/blob/main/Meet_for_Drinks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

To run this notebook, you’ll need to create a [Replicate](https://replicate.com) account and install the Replicate python client.

In [ ]:
# install replicate client
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


# Run and then paste in your API code

In [ ]:
# get a token: https://replicate.com/account
from getpass import getpass
import os
from google.colab import output


REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


Functions to manage the length of prompts

In [ ]:
#Format the prompt message for Llama2
def createMessage(messages):
  message = ""
  for x in messages:
    if x["isUser"]:
      message+="[INST]"+x["text"]+"[/INST]"
    else:
      message+=x["text"]
  return message

#Delete the oldest message
def shrinkMessage(messages):
  del messages[0]

#Make sure the prompt is not over 3000 words or characters?
def messagesSize(messages):
  messageLength = 0
  for x in messages:
    messageLength+=len(x["text"].split())
  return messageLength

## Responses can be slow - as at times it needs to boot up an instance to reply (3-5 mins).

**2023-12-04:**
**V1.1:** Fixed sent prompts having duplicate information


---


**TODO:**




In [ ]:
import replicate
#Speak in French.
system_prompt = """
We are friends and I am calling you to see if you want to go to a bar Friday night
Start conversation with:" “Hey, glad that you called. What is the plan this weekend?”".
keep responses short.
Do not suggest a pub unless asked by me.
Do not list the pubs at the start of the conversation or during the conversation, unless asked for a suggestion.
When asked for a suggestion for a pub, just list one.
There are three pubs nearby we can go to"
1:Bar Lupulus on Wellington
2:Barley Mow in Westboro
3:The Observatory at Algonquin College
".
"""
#Max number of tokens(words) that can be sent as the prompt - DO NOT CHANGE
MAX_TOKENS = 3000

#Changable settings for Llama2
max_new_tokens = 128
min_new_tokens = -1
temperature = 0.1
top_p = 0.9
top_k = 50
debug = False

messageHistory = [
     {"text": "Hi",
      "isUser": True}]
new_prompt = 'empty'
while(new_prompt!=''):
  iterator = replicate.run(
  "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
  input={
                              "prompt": createMessage(messageHistory),
                              "system_prompt": system_prompt,
                              "max_new_tokens": max_new_tokens,
                              "min_new_tokens": min_new_tokens,
                              "temperature": temperature,
                              "top_p": top_p,
                              "top_k": top_k,
                              "debug": debug
                          })

  message = ""
  count = 0
  for text in iterator:
    output.clear()
    count+=1
    if(count>=20):
      message+='\n'
      count=0
    message+=text
    print(message)
  new_prompt = input('Enter next prompt: ')
  messageHistory.append({
      "text": message,
      "isUser": False
  });
  messageHistory.append({
      "text": new_prompt,
      "isUser": True
  });

  while(messagesSize(messageHistory)>MAX_TOKENS):
    shrinkMessage(messageHistory)
  print(messageHistory)






 Awesome! There are three pubs nearby we can go to. Would you like me
 to suggest one, or do you have a preference?


KeyboardInterrupt: Interrupted by user